In [2]:
import pandas as pd
import fasttext
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [3]:
# 读取训练集并转换为FastText需要的格式
df = pd.read_csv('./data/train_set.csv', sep='\t')
train_df = df.loc[:150000]
val_df = df.loc[150000:]
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df.to_csv('./data/train_fasttext.csv', index=None, header=None, sep='\t')

/home/yahu/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
# 训练模型
model = fasttext.train_supervised('./data/train_fasttext.csv',  lr=1.0, wordNgrams=3,
                                  verbose=2, minCount=1, epoch=50, loss="hs")

In [7]:
val_df['label_ft'] = '__label__' + val_df['label'].astype(str)
val_pred = [model.predict(x)[0][0].split('__')[-1] for x in val_df['text']]
print(f1_score(val_df['label'].values.astype(str), val_pred, average='macro'))

/home/yahu/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.8972339984915586


fasttext模型拟合速度快， 效果也不错， 因此用其本地验证结果作为基准。
其验证集上的f1-score为0.8972

In [8]:
# 预测测试集的标签
test_df = pd.read_csv('./data/test_a.csv', sep='\t')
test_pred = [model.predict(x)[0][0].split('__')[-1] for x in test_df['text']]
# 将测试结果保存到文件中
test_pred = pd.Series(test_pred, name='label', dtype='int8')
test_pred.to_csv('./data/submit_ft.csv', index=False)
# 保存模型
model.save_model('./data/fasttext.h5')

/home/yahu/anaconda3/envs/ml/lib/python3.7/site-packages/numpy/core/numeric.py:2378: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
